In [1]:
from sentinelhub import (
    SHConfig,
    BBox,
    bbox_to_dimensions,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    BBoxSplitter
)

import geojson
import folium
import math

In [26]:
def get_image(coordinates):
    
    # put here your sentinel hub credential
    config = SHConfig()
    config.sh_client_id = "95d58b19-89f3-4e67-b4b5-d52bc0e686e6"
    config.sh_client_secret = "unywHIATbGmqvHztJkWMbMyXSMqVSQkB"

    # customize this functions to specify the band you want to download
    evalscript = """
        // VERSION=3
        
        function setup() {
            return {
                input: [{
                    bands: [
                        "B08",
                        "B11"
                    ],
                }],
                output: {
                    bands: 2,
                }
            };
        }

        function evaluatePixel(sample) {
            return [
                sample.B08,
                sample.B11
            ];
        }
    """

    # put here the coordinate of the image you want to download
    # coordinates = (-96.455, 41.146, -96.450, 41.153)

    # set the coordinate reference system
    crs = 4326

    # set the desired resolution
    resolution = 10

    # set the time interval
    time_interval = ("2023-08-01", "2023-08-31")

    # set the source of the image
    data_collection = DataCollection.SENTINEL2_L2A

    # set the constant class for type of mosaicking order
    mosaicking_order = MosaickingOrder.LEAST_CC

    # update this value if you want to save the image
    save_data = True

    # set the folder where you want to dowload the image
    data_folder = './remaining_rasters'

    # set the format of the image
    response_format = MimeType.TIFF

    bbox = BBox(bbox = coordinates, crs = crs)
    size = bbox_to_dimensions(bbox, resolution = resolution)
    request = SentinelHubRequest(
        config = config,
        bbox = bbox,
        size = size,
        evalscript = evalscript,
        data_folder = data_folder,
        input_data = [
            SentinelHubRequest.input_data(
                time_interval = time_interval,
                data_collection = data_collection,
                mosaicking_order = mosaicking_order
            )
        ],
        responses = [
            SentinelHubRequest.output_response(
                identifier = "default",
                response_format = response_format
            )
        ]
    )

    return request.get_data(save_data=save_data)

In [27]:
def divide_bbox(bbox:BBox, resolution:float, max_size=2400):
    dimensions = bbox_to_dimensions(bbox, resolution)
    if dimensions[0] > max_size or dimensions[1] > max_size:
        x_divisions = max(1, math.ceil(dimensions[0] / max_size))
        y_divisions = max(1, math.ceil(dimensions[1] / max_size))
        bbox_splitter = BBoxSplitter([bbox], 4326, (x_divisions, y_divisions))
        return bbox_splitter.get_bbox_list()
    else:
        return [bbox]

left = 180
bottom = 90
right = -180
top = -90

map = folium.Map(location=[-10, -42], zoom_start=7)

with open(f'../../veg-monitor/notebooks/data/ecuatorial/ecuatorial_petrolina_tower_dataframe.geojson') as f:
    dataframe = geojson.load(f)

for powerline in dataframe['features']:
    x = powerline['geometry']['coordinates'][0]
    y = powerline['geometry']['coordinates'][1]
    if x < left:
        left = x
    if x > right:
        right = x
    if y < bottom:
        bottom = y
    if y > top:
        top = y

bbox_list = []
bbox_list_no = []

for bbox in divide_bbox(BBox((left, bottom, right, top), 4326), 10):
    left, bottom, right, top = bbox
    presente = False
    for powerline in dataframe['features']:
        x = powerline['geometry']['coordinates'][0]
        y = powerline['geometry']['coordinates'][1]
        if left < x and x < right and bottom < y and y < top:
            presente = True
    if presente:
        bbox_list.append(bbox)

for bbox in bbox_list:
    left, bottom, right, top = bbox
    folium.Rectangle([(top, left), (bottom, right)], color='black', weight=1).add_to(map)

bbox_list_no = [
    BBox((-43.3789235,-9.7960732,-43.161923,-9.5949815), 4326),
    BBox((-43.161923,-9.997164900000001,-42.9449225,-9.7960732), 4326),
    BBox((-41.85992,-9.192798100000001,-41.6429195,-8.9917064), 4326),
]

for bbox in bbox_list_no:
    left, bottom, right, top = bbox
    folium.Rectangle([(top, left), (bottom, right)], color='red', weight=1).add_to(map)

for bbox in bbox_list_no:
    image = get_image(bbox)

map


c:\Users\EBELLIMTP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentinelhub\geometry.py:137: SHDeprecationWarning: Initializing `BBox` objects from `BBox` objects will no longer be possible in future versions.
  return cls._tuple_from_bbox(bbox)
